In [ ]:
!pip install -q transformers datasets evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset

# Cargar datos
df = pd.read_csv("Corpus_estres.csv")
df['label'] = df['Estres'].map({'si': 1, 'no': 0})

# División 80/20
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=0)

# Convertir a HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df[['Transcripcion', 'label']])
test_dataset = Dataset.from_pandas(test_df[['Transcripcion', 'label']])


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# BETO: preentrenado para español
model_name = "dccuchile/bert-base-spanish-wwm-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(batch):
    return tokenizer(batch["Transcripcion"], padding=True, truncation=True)

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/486 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate

# Métricas
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

# Parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results_beto",
    do_train=True,
    do_eval=True,
    report_to="none",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs_beto",
    logging_steps=20,
    save_steps=100,
    eval_steps=20,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-11-1372589859.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
20,0.207500
40,0.214800
60,0.048400
80,0.032500


TrainOutput(global_step=98, training_loss=0.1179215068719825, metrics={'train_runtime': 55.0413, 'train_samples_per_second': 17.659, 'train_steps_per_second': 1.78, 'total_flos': 119879974598400.0, 'train_loss': 0.1179215068719825, 'epoch': 2.0})

In [ ]:
eval_result = trainer.evaluate()
print(eval_result)


{'eval_loss': 0.8053984045982361, 'eval_accuracy': 0.8360655737704918, 'eval_f1_macro': 0.8344640434192674, 'eval_runtime': 0.6981, 'eval_samples_per_second': 174.764, 'eval_steps_per_second': 18.622, 'epoch': 2.0}


roberta con lora


In [ ]:
!pip install peft transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate

# Cargar datos
df = pd.read_csv("Corpus_estres.csv")
df['label'] = df['Estres'].map({'si': 1, 'no': 0})

# División 80/20
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=0)

# Convertir a Dataset de HuggingFace
train_dataset = Dataset.from_pandas(train_df[['Transcripcion', 'label']])
test_dataset = Dataset.from_pandas(test_df[['Transcripcion', 'label']])

# Tokenizador y modelo base
model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Configuración de LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Clasificación de secuencias
    inference_mode=False,
    r=8,                        # rango bajo
    lora_alpha=32,
    lora_dropout=0.1
)

# Aplicar LoRA al modelo base
model = get_peft_model(base_model, lora_config)

# Tokenización
def tokenize(batch):
    return tokenizer(batch["Transcripcion"], padding=True, truncation=True)

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

# Métricas
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results_beto_lora",
    do_train=True,
    do_eval=True,
    report_to="none",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=2,
    learning_rate=2e-4,  # Puede ser un poco mayor con LoRA
    weight_decay=0.01,
    logging_dir="./logs_beto_lora",
    logging_steps=20,
    save_steps=100,
    eval_steps=20,
    save_total_limit=2
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenamiento
trainer.train()

# Evaluación
eval_result = trainer.evaluate()
print(eval_result)


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

...

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

/tmp/ipython-input-6-1760859213.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,0.661200
40,0.588700
60,0.406800
80,0.325700


{'eval_loss': 0.4021609425544739, 'eval_accuracy': 0.860655737704918, 'eval_f1_macro': 0.860195483653522, 'eval_runtime': 0.6178, 'eval_samples_per_second': 197.461, 'eval_steps_per_second': 21.041, 'epoch': 2.0}


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate

# Cargar datos
df = pd.read_csv("Corpus_estres.csv")
df['label'] = df['Estres'].map({'si': 1, 'no': 0})

# División 80/20
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=0)

# Convertir a Dataset de HuggingFace
train_dataset = Dataset.from_pandas(train_df[['Transcripcion', 'label']])
test_dataset = Dataset.from_pandas(test_df[['Transcripcion', 'label']])

# Tokenizador y modelo base
model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Configuración de LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Clasificación de secuencias
    inference_mode=False,
    r=16,                        # rango bajo
    lora_alpha=128,
    lora_dropout=0.2
)

# Aplicar LoRA al modelo base
model = get_peft_model(base_model, lora_config)

# Tokenización
def tokenize(batch):
    return tokenizer(batch["Transcripcion"], padding=True, truncation=True)

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

# Métricas
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results_roberta_lora",
    do_train=True,
    do_eval=True,
    report_to="none",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=4e-4,  # Puede ser un poco mayor con LoRA
    weight_decay=0.01,
    logging_dir="./logs_roberta_lora",
    logging_steps=20,
    save_steps=100,
    eval_steps=20,
    save_total_limit=2
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenamiento
trainer.train()

# Evaluación
eval_result = trainer.evaluate()
print(eval_result)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/486 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,0.577500
40,0.353600
60,0.292300
80,0.164500


{'eval_loss': 0.40406179428100586, 'eval_accuracy': 0.8688524590163934, 'eval_f1_macro': 0.8682860998650472, 'eval_runtime': 0.5096, 'eval_samples_per_second': 239.425, 'eval_steps_per_second': 15.7, 'epoch': 3.0}


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
from sklearn.model_selection import train_test_split

# 1️⃣ Cargar datos
df = pd.read_csv("Corpus_estres.csv")
df['label'] = df['Estres'].map({'si': 1, 'no': 0})

# 2️⃣ Dividir
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=0)
train_dataset = Dataset.from_pandas(train_df[['Transcripcion', 'label']])
test_dataset = Dataset.from_pandas(test_df[['Transcripcion', 'label']])

# 3️⃣ Tokenizador y modelo base (RoBERTa español)
model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 4️⃣ Configuración LoRA (mayor capacidad)
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=32,
    lora_alpha=64,
    lora_dropout=0.1
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# 5️⃣ Tokenización
def tokenize(batch):
    return tokenizer(batch["Transcripcion"], padding=True, truncation=True)

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

# 6️⃣ Métricas
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

# 7️⃣ Argumentos de entrenamiento optimizados
training_args = TrainingArguments(
    output_dir="./results_roberta_lora",
    do_train=True,
    do_eval=True,
    report_to="none",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=2e-4,
    weight_decay=0.05,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_dir="./logs_roberta_lora",
    logging_steps=10,
    save_steps=50,
    eval_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    eval_strategy="steps" # Added this line to match save_strategy
)

# 8️⃣ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 9️⃣ Entrenamiento
trainer.train()

# 🔟 Evaluación final
eval_result = trainer.evaluate()
print(eval_result)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,771,778 || all params: 126,416,644 || trainable%: 1.4015


Map:   0%|          | 0/486 [00:00<?, ? examples/s]

Map:   0%|          | 0/122 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy,F1 Macro
50,0.464100,0.510051,0.762295,0.762151
100,0.284100,0.375139,0.836066,0.835668
150,0.175900,0.459125,0.852459,0.852101
200,0.092800,0.599625,0.795082,0.795068
250,0.134800,0.582153,0.836066,0.835889
300,0.062000,0.594761,0.827869,0.827765


{'eval_loss': 0.45912477374076843, 'eval_accuracy': 0.8524590163934426, 'eval_f1_macro': 0.8521012931034483, 'eval_runtime': 0.5978, 'eval_samples_per_second': 204.077, 'eval_steps_per_second': 26.764, 'epoch': 5.0}
